# Microarray Data Aanlysis

# 1. Environment

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import session_info

In [2]:
session_info.show()

# 2. Data Processing

In [3]:
result_mi = "results"
if not os.path.exists(result_mi):
    os.makedirs(result_mi)

In [4]:
B16mRNA_path = "GSE57304_RAW"

In [5]:
files_mRNA = os.listdir(B16mRNA_path)
print(files_mRNA)

['GSM1379344_N_mix.txt', 'GSM1379335_E_mix.txt', 'GSM1379336_F_mix.txt', 'GSM1379338_H_mix.txt', 'GSM1379343_M_mix.txt', 'GSM1379340_J_mix.txt', 'GSM1379332_B_mix.txt', 'GSM1379331_A_mix.txt', 'GSM1379334_D_mix.txt', 'GSM1379339_I_mix.txt', 'GSM1379333_C_mix.txt', 'GSM1379337_G_mix.txt', 'GSM1379342_L_mix.txt', 'GSM1379341_K_mix.txt']


In [6]:
# Mapping between file names and experimental conditions
files_conditions = {
    "GSM1379331_A_mix.txt": "CTL-d1",
    "GSM1379332_B_mix.txt": "Untreated-d1",
    "GSM1379333_C_mix.txt": "CTL-d3",
    "GSM1379334_D_mix.txt": "Untreated-d3",
    "GSM1379335_E_mix.txt": "CTL-d3-aIFNg",
    "GSM1379336_F_mix.txt": "CTL-d3-IgG",
    "GSM1379337_G_mix.txt": "CTL-d5",
    "GSM1379338_H_mix.txt": "Untreated-d5",
    "GSM1379339_I_mix.txt": "CTL-d5-aIFNg",
    "GSM1379340_J_mix.txt": "CTL-d5-IgG",
    "GSM1379341_K_mix.txt": "CTL-d7",
    "GSM1379342_L_mix.txt": "Untreated-d7",
    "GSM1379343_M_mix.txt": "CTL-d7-aIFNg",
    "GSM1379344_N_mix.txt": "CTL-d7-IgG",
}

In [7]:
common_columns = ["FeatureNum", "ProbeName", "GeneName"]
select_columns = ["gIsWellAboveBG", "ControlType", "gProcessedSignal"]

# List to store individual data frames
data_frames_list = []

# Load and filter each file
for file, condition in files_conditions.items():
    df = pd.read_csv(os.path.join(B16mRNA_path, file), sep="\t", header=9)
    
    # Select only necessary columns
    df = df[common_columns + select_columns]

    # Rename columns: add condition suffix to gIsWellAboveBG and rename gProcessedSignal to condition
    df = df.rename(columns={"gProcessedSignal": condition, "gIsWellAboveBG": f"gIsWellAboveBG_{condition}"})
    df = df.drop(columns=["ControlType"])

    data_frames_list.append(df)

# Initialize merged dataframe with common columns
merged_df = data_frames_list[0][common_columns]

# Merge processed signal values for all conditions
for df in data_frames_list:
    merged_df = pd.merge(merged_df, df.drop(columns=[col for col in df.columns if "gIsWellAboveBG" in col]), on=common_columns, how='outer')

# Merge gIsWellAboveBG columns for all conditions
for df in data_frames_list:
    merged_df = pd.merge(merged_df, df[common_columns + [col for col in df.columns if "gIsWellAboveBG" in col]], on=common_columns, how='outer')

# Remove probes where all gIsWellAboveBG values are 0
gIsWellAboveBG_columns = [col for col in merged_df.columns if "gIsWellAboveBG" in col]
mask = (merged_df[gIsWellAboveBG_columns] != 0).any(axis=1)
filtered_df = merged_df[mask]

# Drop gIsWellAboveBG columns as they are no longer needed
filtered_df = filtered_df.drop(columns=gIsWellAboveBG_columns)

In [8]:
filtered_df

,FeatureNum,ProbeName,GeneName,CTL-d1,Untreated-d1,CTL-d3,Untreated-d3,CTL-d3-aIFNg,CTL-d3-IgG,CTL-d5,Untreated-d5,CTL-d5-aIFNg,CTL-d5-IgG,CTL-d7,Untreated-d7,CTL-d7-aIFNg,CTL-d7-IgG
0,1,GE_BrightCorner,GE_BrightCorner,72816.550000,72574.840000,57762.220000,52167.150000,49276.50000,53071.530000,87514.300000,68143.860000,62584.980000,77984.640000,70392.270000,36632.630000,48600.370000,64915.200000
11,12,A_52_P616356,Ccr1,6.212454,6.259565,4.941514,5.114783,86.16759,6.080089,7.831572,5.756872,5.039113,5.527968,4.237431,3.246883,4.453829,5.171262
12,13,A_52_P580582,Nppa,33.682120,41.097460,42.110610,44.763300,26.56708,45.522710,76.817890,54.720000,31.964290,59.521090,50.197730,92.496780,48.837290,38.492520
15,16,A_51_P331831,Hvcn1,3066.995000,2734.202000,948.542800,2546.954000,2176.20800,1287.180000,1236.777000,2219.893000,2697.986000,1513.148000,1372.217000,1448.382000,2715.330000,1583.121000
18,20,A_52_P299964,Maml2,20.135550,16.219350,19.285140,14.279600,10.66470,30.730820,17.744130,12.789320,8.190339,18.135410,7.336438,10.056680,13.753590,14.726800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45003,45206,A_52_P189897,Gtf2a2,6304.722000,5848.598000,2603.145000,5624.848000,4559.01700,3408.235000,2383.762000,6562.064000,5555.188000,3220.067000,2374.960000,3663.912000,6348.717000,3753.290000
45004,45207,A_52_P37634,D11Wsu47e,2185.912000,2048.473000,1349.070000,2079.274000,1435.11800,1655.758000,1049.264000,1918.193000,1617.600000,1649.320000,1074.846000,1084.544000,1856.098000,1238.686000
45006,45209,A_52_P980518,AK078781,49.245450,45.962440,16.320720,39.876760,37.74634,27.729620,12.654900,58.440170,40.330510,17.688640,9.826316,30.309980,45.482910,26.222770
45016,45219,GE_BrightCorner,GE_BrightCorner,73343.070000,68887.020000,58154.400000,48154.310000,50316.23000,47690.810000,90953.810000,81104.440000,68054.900000,79665.120000,65077.840000,36144.470000,49418.750000,59578.480000


In [9]:
output_file = "filtered_data.csv"
filtered_df.to_csv(os.path.join(result_mi, output_file), index=False)

# 3. Normalization using the 75th percentile method

In [10]:
# Select columns corresponding to gProcessedSignal values
x = filtered_df.iloc[:, 3:]  # assuming first 3 columns are metadata

# Compute the 75th percentile for each column
q75 = x.quantile(0.75)

# Normalize each column by its 75th percentile
x_norm = x.div(q75, axis=1)

In [11]:
# GeneNameとFeatureNumを追加
x_norm['Gene'] = filtered_df['GeneName']
x_norm['FeatureNum'] = filtered_df['FeatureNum']

In [12]:
x_norm.head()

,CTL-d1,Untreated-d1,CTL-d3,Untreated-d3,CTL-d3-aIFNg,CTL-d3-IgG,CTL-d5,Untreated-d5,CTL-d5-aIFNg,CTL-d5-IgG,CTL-d7,Untreated-d7,CTL-d7-aIFNg,CTL-d7-IgG,Gene,FeatureNum
0,59.710952,65.834237,74.787133,49.201435,56.029647,57.331997,128.711393,61.273892,62.072417,87.747798,108.585917,54.310097,44.801061,80.992968,GE_BrightCorner,1
11,0.005094,0.005678,0.006398,0.004824,0.097977,0.006568,0.011518,0.005176,0.004998,0.006220,0.006537,0.004814,0.004106,0.006452,Ccr1,12
12,0.027620,0.037280,0.054522,0.042218,0.030208,0.049177,0.112980,0.049203,0.031703,0.066973,0.077434,0.137132,0.045019,0.048026,Nppa,13
15,2.514994,2.480255,1.228118,2.402159,2.474449,1.390512,1.818986,1.996093,2.675890,1.702584,2.116759,2.147314,2.503060,1.975218,Hvcn1,16
18,0.016512,0.014713,0.024969,0.013468,0.012126,0.033198,0.026097,0.011500,0.008123,0.020406,0.011317,0.014910,0.012678,0.018374,Maml2,20


In [13]:
norm_output_csv = "normalized_data.csv"
x_norm.to_csv(os.path.join(result_mi, norm_output_csv), index=False)

# 4. Selection of genes of interest

In [14]:
# List of genes to analyze
genes_of_interest = ["Ifng", "Gzmb", "Prf1", "Fas", "Fasl", "Cd8a", "Stat1", "Cxcl9", "Cxcl10", "Icam1"]

# List to store mean and standard deviation for each gene-condition pair
summary_stats_list = []

# Compute mean and standard deviation for each gene under each condition
for gene in genes_of_interest:
    for condition in files_conditions.values():
        mask = (x_norm["Gene"] == gene)
        mean_value = x_norm.loc[mask, condition].mean()
        stddev_value = x_norm.loc[mask, condition].std()
        summary_stats_list.append({
            "Gene": gene,
            "Experiment": condition,
            "Mean": mean_value,
            "StdDev": stddev_value
        })

# Convert the summary list into a DataFrame
summary_stats = pd.DataFrame(summary_stats_list)

In [15]:
summary_stats.head()

,Gene,Experiment,Mean,StdDev
0,Ifng,CTL-d1,0.027729,0.011232
1,Ifng,Untreated-d1,0.020618,0.037885
2,Ifng,CTL-d3,1.755238,0.449217
3,Ifng,Untreated-d3,0.006834,0.004162
4,Ifng,CTL-d3-aIFNg,0.287980,0.080480


## Mapping

In [16]:
# Create 'Condition' and 'Day' columns using the mapping dictionary
condition_mapping = {
    'CTL-d1': ('CTL', 1),
    'CTL-d3': ('CTL', 3),
    'CTL-d5': ('CTL', 5),
    'CTL-d7': ('CTL', 7),
    'Untreated-d1': ('Untreated', 1),
    'Untreated-d3': ('Untreated', 3),
    'Untreated-d5': ('Untreated', 5),
    'Untreated-d7': ('Untreated', 7),
    'CTL-d3-aIFNg': ('CTL+anti-IFNg', 3),
    'CTL-d5-aIFNg': ('CTL+anti-IFNg', 5),
    'CTL-d7-aIFNg': ('CTL+anti-IFNg', 7),
    'CTL-d3-IgG': ('CTL+RatIgG', 3),
    'CTL-d5-IgG': ('CTL+RatIgG', 5),
    'CTL-d7-IgG': ('CTL+RatIgG', 7)
}

In [17]:
def map_condition(variable):
    treatment, day = condition_mapping.get(variable, (None, None))
    return pd.Series([treatment, day], index=['treatment', 'day'])

In [18]:
# Apply mapping to create new columns
summary_stats[['Condition', 'Day']] = summary_stats['Experiment'].apply(lambda x: pd.Series(condition_mapping[x]))

In [19]:
summary_stats

,Gene,Experiment,Mean,StdDev,Condition,Day
0,Ifng,CTL-d1,0.027729,0.011232,CTL,1
1,Ifng,Untreated-d1,0.020618,0.037885,Untreated,1
2,Ifng,CTL-d3,1.755238,0.449217,CTL,3
3,Ifng,Untreated-d3,0.006834,0.004162,Untreated,3
4,Ifng,CTL-d3-aIFNg,0.287980,0.080480,CTL+anti-IFNg,3
...,...,...,...,...,...,...
135,Icam1,CTL-d5-IgG,0.488984,0.147989,CTL+RatIgG,5
136,Icam1,CTL-d7,0.328201,0.103439,CTL,7
137,Icam1,Untreated-d7,0.065868,0.020655,Untreated,7
138,Icam1,CTL-d7-aIFNg,0.073778,0.023449,CTL+anti-IFNg,7


In [20]:
# Export the summary statistics (mean and standard deviation) to an Excel file
summary_output_file = "cytokine_statistics.csv"
summary_stats.to_csv(os.path.join(result_mi, summary_output_file), index=False)